# StyleGAN3 Training — Latent Resonance Spectrograms (Kaggle)

Train StyleGAN3 on 512x512 grayscale spectrogram images.

**Setup:** In the Kaggle sidebar, go to **Settings → Accelerator → GPU T4 x2**.

**Dataset:** Upload your `spectrograms.zip` as a [Kaggle Dataset](https://www.kaggle.com/datasets),
then add it to this notebook via **Add data** in the sidebar.

## 1. Setup & GPU Check

In [ ]:
!nvidia-smi
!pip install -q ninja

import torch
assert torch.cuda.is_available(), "No GPU — enable it in Settings → Accelerator → GPU T4 x2"
print(f"PyTorch {torch.__version__}, CUDA {torch.version.cuda}, GPU: {torch.cuda.get_device_name(0)}")

## 2. Clone StyleGAN3 & Apply Patches

In [ ]:
import os
import sys
import pathlib
import shutil
import subprocess

# Fresh clone
if os.path.exists("stylegan3"):
    shutil.rmtree("stylegan3")
!git clone https://github.com/NVlabs/stylegan3.git
sys.path.insert(0, "stylegan3")

# ── Patch 1: Fix InfiniteSampler for PyTorch ≥2.4 ────────────────────────
misc_path = pathlib.Path("stylegan3/torch_utils/misc.py")
src = misc_path.read_text()
src = src.replace("super().__init__(dataset)", "super().__init__()")
misc_path.write_text(src)
print(f"Patched {misc_path}: InfiniteSampler fix")

# ── Patch 2: Shape mismatch grayscale and RGB ────────────────
lines = src.split('\n')
new_lines = []
patched = False
for line in lines:
    if 'tensor.copy_(src_tensors[name].detach())' in line and not patched:
        indent = line[:len(line) - len(line.lstrip())]
        new_lines.append(f'{indent}if src_tensors[name].shape != tensor.shape:')
        new_lines.append(f'{indent}    continue')
        new_lines.append(line)
        patched = True
    else:
        new_lines.append(line)
src = '\n'.join(new_lines)
assert patched, "ERROR: could not patch copy_params_and_buffers"
print(f"  Patch 2: shape-mismatch guard {'applied' if patched else 'FAILED'}")

misc_path.write_text(src)
print(f"Patched {misc_path}")

# ── Patch 3: Fix Adam betas int → float for PyTorch ≥2.9 ────────────────
train_path = pathlib.Path("stylegan3/train.py")
src = train_path.read_text()
src = src.replace("betas=[0,0.99]", "betas=[0.0,0.99]")
train_path.write_text(src)
print(f"Patched {train_path}: Adam betas fix")

# ── Patch 4: Try CUDA ops compilation ────────────────────────────────────
# Detect GPU compute capability automatically
cc_major, cc_minor = torch.cuda.get_device_capability(0)
arch = f"{cc_major}.{cc_minor}"
os.environ["TORCH_CUDA_ARCH_LIST"] = arch
os.environ["TORCH_EXTENSIONS_DIR"] = "/tmp/torch_extensions"
if os.path.exists("/tmp/torch_extensions"):
    shutil.rmtree("/tmp/torch_extensions")

result = subprocess.run(
    ["python", "-c",
     "import sys; sys.path.insert(0,'stylegan3'); "
     "from torch_utils.ops import bias_act; "
     "assert bias_act._init(), 'init failed'"],
    capture_output=True, text=True, timeout=180,
)

CUDA_OPS_OK = result.returncode == 0
if CUDA_OPS_OK:
    print(f"Custom CUDA ops compiled for sm_{cc_major}{cc_minor} — using fused kernels")
else:
    print(f"CUDA ops compilation failed (arch {arch}), using native PyTorch fallback")
    print(f"  Error: ...{result.stderr[-300:]}")
    ops_dir = pathlib.Path("stylegan3/torch_utils/ops")
    for name in ["bias_act.py", "upfirdn2d.py", "filtered_lrelu.py"]:
        p = ops_dir / name
        s = p.read_text()
        s = s.replace("def _init():", "def _init():\n    return False")
        p.write_text(s)
        print(f"  Patched {p}")

## 3. Load Dataset

Kaggle datasets are mounted at `/kaggle/input/<dataset-name>/`.

Set `KAGGLE_DATASET` to match your dataset name.

In [ ]:
import os
import glob
import shutil

KAGGLE_DATASET = "spectrograms"  # <-- your Kaggle dataset name

input_dir = f"/kaggle/input/{KAGGLE_DATASET}"

# Find PNGs (may be in root or a subfolder)
pngs = glob.glob(f"{input_dir}/**/*.png", recursive=True)

# Copy to a writable working directory (Kaggle input is read-only)
DATASET_PATH = "/kaggle/working/spectrograms"
os.makedirs(DATASET_PATH, exist_ok=True)
for p in pngs:
    shutil.copy(p, DATASET_PATH)

print(f"Found {len(pngs)} PNG files → copied to {DATASET_PATH}")

## 4. Prepare Dataset

In [ ]:
!python stylegan3/dataset_tool.py \
    --source={DATASET_PATH} \
    --dest=./spectrograms.zip \
    --resolution=512x512

## 5. Configure Training

In [ ]:
import os
os.environ["PYTORCH_ALLOC_CONF"] = "expandable_segments:True"

GPUS = 2
GAMMA = 2.0
SNAP = 10
KIMG = 400
CFG = "stylegan2"
MIRROR = False
METRICS = "none"
AUG= "ada"
RESUME = "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-ffhq-512x512.pkl"
# RESUME = ""
if CUDA_OPS_OK:
    BATCH_SIZE = 16
    CBASE = 16384
    CMAX = 512
    MBSTD = 8
    print(f"CUDA ops available — batch={BATCH_SIZE}, full model capacity")
else:
    BATCH_SIZE = 2
    CBASE = 8192
    CMAX = 128
    MBSTD = 2
    print(f"Native fallback — batch={BATCH_SIZE}, cbase={CBASE}, cmax={CMAX}")

## 6. Train

In [ ]:
import torch
torch.cuda.empty_cache()

resume_flag = f"--resume={RESUME}" if RESUME else ""
mirror_int = 1 if MIRROR else 0

!python stylegan3/train.py \
    --outdir=./training-runs \
    --cfg={CFG} \
    --data=./spectrograms.zip \
    --gpus={GPUS} \
    --batch={BATCH_SIZE} \
    --batch-gpu={int(BATCH_SIZE / GPUS)} \
    --gamma={GAMMA} \
    --snap={SNAP} \
    --kimg={KIMG} \
    --mirror={mirror_int} \
    --metrics={METRICS} \
    --cbase={CBASE} \
    --cmax={CMAX} \
    --mbstd-group={MBSTD} \
    --aug={AUG}\
    {resume_flag}

## 7. Generate Samples

In [ ]:
import glob
import pickle

import matplotlib.pyplot as plt
import torch

pkls = sorted(glob.glob("training-runs/**/*.pkl", recursive=True))
assert pkls, "No snapshots found — has training completed at least one snapshot?"
latest_pkl = pkls[-1]
print(f"Loading {latest_pkl}")

with open(latest_pkl, "rb") as f:
    G = pickle.load(f)["G_ema"].cuda().eval()

z = torch.randn(4, G.z_dim, device="cuda")
with torch.no_grad():
    imgs = G(z, None)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for i, ax in enumerate(axes):
    img = imgs[i, 0].cpu().numpy()
    ax.imshow(img, cmap="magma", aspect="auto")
    ax.set_title(f"Sample {i}")
    ax.axis("off")
plt.suptitle("Generated Spectrograms")
plt.tight_layout()
plt.show()

## 8. Save Results

Kaggle persists everything in `/kaggle/working/` as notebook output.
Click **Save Version** (top right) → the training-runs zip will be available under **Output**.

In [ ]:
import shutil

shutil.make_archive("/kaggle/working/training-runs", "zip", ".", "training-runs")
print("Created /kaggle/working/training-runs.zip")
print("This will be saved as notebook output when you click Save Version.")

In [ ]:
  !ls -la training-runs/                                                                                                   